In [1]:
import pandas as pd

# Корабли

Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.
Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах). В отношении Ships записаны название корабля, имя его класса и год спуска на воду. В отношение Battles включены название и дата битвы, в которой участвовали корабли, а в отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).
Замечания. 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 2) Потопленный корабль в последующих битвах участия не принимает.

![](images/ships.png)

In [2]:
classes = pd.read_csv('data/classes.csv', index_col='class')
classes.head()

,type,country,guns,bore,displacement
class,,,,,
Bismarck,bb,Germany,8,15.0,42000
Iowa,bb,USA,9,16.0,46000
Kongo,bc,Japan,8,14.0,32000
North Carolina,bb,USA,12,16.0,37000
Renown,bc,Gt.Britain,6,15.0,32000


In [3]:
ships = pd.read_csv('data/ships.csv', index_col='name')
ships.head()

,class,launched
name,,
California,Tennessee,1921
Haruna,Kongo,1916
Hiei,Kongo,1914
Iowa,Iowa,1943
Kirishima,Kongo,1915


In [4]:
outcomes = pd.read_csv('data/outcomes.csv', index_col=['ship', 'battle'])
outcomes.head()

,,result
ship,battle,
Bismarck,North Atlantic,sunk
California,Surigao Strait,OK
Duke of York,North Cape,OK
Fuso,Surigao Strait,sunk
Hood,North Atlantic,sunk


In [5]:
battles = pd.read_csv('data/battles.csv', index_col='name')
battles.head()

,date
name,
Guadalcanal,1942-11-15 00:00:00
North Atlantic,1941-05-25 00:00:00
North Cape,1943-12-26 00:00:00
Surigao Strait,1944-10-25 00:00:00


# Задача 1.

Найдите страны, имевшие когда-либо классы обычных боевых кораблей ('bb') и имевшие когда-либо классы крейсеров ('bc').

![](images/ships.png)

### Pandas

Без использования дополнительных переменных. Искомые страны - те, для которых type == 2.

In [38]:
classes[['country', 'type']].groupby(['country']).nunique()

,country,type
country,,
Germany,1,1
Gt.Britain,1,2
Japan,1,2
USA,1,1


С использованием дополнительной переменной, но явным выводом названий искомых стран.

In [37]:
result_countries = classes[['country', 'type']].groupby(['country']).nunique()
result_countries = result_countries.drop(columns=['country'])
result_countries[result_countries['type'] == 2].reset_index()[['country']]

,country
0,Gt.Britain
1,Japan


# Задача 2.

Для каждого класса определите год, когда был спущен на воду первый корабль этого из класса.
![](images/ships.png)

### Pandas

In [49]:
# Воспользуйтесь merge, groupby, min
# Попробуйте решить задачу одной командой без введения дополнительных переменных
classes.reset_index().merge(ships, how='left', on='class').groupby('class').min()[['launched']]

,launched
class,
Bismarck,NaN
Iowa,1943.0
Kongo,1913.0
North Carolina,1941.0
Renown,1916.0
Revenge,1916.0
Tennessee,1920.0
Yamato,1941.0


# Самолёты

Таблица company содержит идентификатор и название компании, осуществляющей перевозку пассажиров. Таблица trip содержит информацию о рейсах: номер рейса, идентификатор компании, тип самолета, город отправления, город прибытия, время отправления и время прибытия. Таблица passenger содержит идентификатор и имя пассажира. Таблица pass_in_trip содержит информацию о полетах: номер рейса, дата вылета (день), идентификатор пассажира и место, на котором он сидел во время полета.


![](images/planes.png)

In [50]:
trip = pd.read_csv('data/trip.csv', index_col='trip_no',
    parse_dates=['time_out', 'time_in'], infer_datetime_format=True)
trip.head()

,id_comp,plane,town_from,town_to,time_out,time_in
trip_no,,,,,,
1100,4,Boeing,Rostov,Paris,1900-01-01 14:30:00,1900-01-01 17:50:00
1101,4,Boeing,Paris,Rostov,1900-01-01 08:12:00,1900-01-01 11:45:00
1123,3,TU-154,Rostov,Vladivostok,1900-01-01 16:20:00,1900-01-02 03:40:00
1124,3,TU-154,Vladivostok,Rostov,1900-01-01 09:00:00,1900-01-01 19:50:00
1145,2,IL-86,Moscow,Rostov,1900-01-01 09:35:00,1900-01-01 11:23:00


In [51]:
pass_in_trip = pd.read_csv('data/pass_in_trip.csv',
    infer_datetime_format=True, parse_dates=['date'])
pass_in_trip.head()

,trip_no,date,id_psg,place
0,1100,2003-04-29,1,1a
1,1123,2003-04-05,3,2a
2,1123,2003-04-08,1,4c
3,1123,2003-04-08,6,4b
4,1124,2003-04-02,2,2d


In [52]:
company = pd.read_csv('data/company.csv', index_col='id_comp')
company.head()

,name
id_comp,
1,Don_avia
2,Aeroflot
3,Dale_avia
4,air_France
5,British_AW


In [53]:
passenger = pd.read_csv('data/passenger.csv', index_col='id_psg')
passenger.head()

,name
id_psg,
1,Bruce Willis
2,George Clooney
3,Kevin Costner
4,Donald Sutherland
5,Jennifer Lopez


# Задача 3.

Определить имена пассажиров, когда-либо летевших на одном и том же месте более одного раза. 

![](images/planes.png)

### Pandas

Искомые имена - те, для которых flag == True

In [158]:
# Попробуйте решить одной командой используя один merge и два groupby, функции count и sum

(pass_in_trip.groupby(['id_psg', 'place']).count()['trip_no'] > 1).groupby(['id_psg']).sum().to_frame()\
    .merge(passenger, on='id_psg').rename(columns={'trip_no': 'flag'}).set_index('name')

,flag
name,
Bruce Willis,True
George Clooney,False
Kevin Costner,False
Jennifer Lopez,False
Ray Liotta,False
Nikole Kidman,True
Alan Rickman,False
Kurt Russell,False
Harrison Ford,False


Вариант с apply (остаются только искомые имена):

In [240]:
(pass_in_trip.groupby(['id_psg', 'place']).count()['trip_no'] > 1).groupby(['id_psg']).sum()\
    .to_frame().apply(lambda x: x[x])\
    .merge(passenger, on='id_psg')[['name']]

,name
id_psg,
1,Bruce Willis
8,Nikole Kidman
37,Mullah Omar


# Задача 4.

Для каждой авиакомпании определить:

* количество выполненных перелетов;
* число использованных типов самолетов;
* количество перевезенных различных пассажиров;
* общее число перевезенных компанией пассажиров. 

![](images/planes.png)

### Pandas

In [134]:
trip.reset_index().groupby(['id_comp']).nunique()[['trip_no', 'plane']]\
    .merge(company, left_on='id_comp', right_index=True)\
    .rename(columns={'trip_no': 'flights_num', 'plane': 'planes_num'}).set_index('name')

,flights_num,planes_num
name,,
Don_avia,6,2
Aeroflot,2,1
Dale_avia,2,1
air_France,2,1
British_AW,10,1


In [157]:
trip.reset_index().merge(pass_in_trip, on='trip_no').groupby('id_comp').agg(['nunique', 'size'])['id_psg']\
    .merge(company, left_index=True, right_on='id_comp').set_index('name')\
    .rename(columns={'nunique': 'num_unique_psg', 'size': 'num_psg'})

,num_unique_psg,num_psg
name,,
Don_avia,6,9
Aeroflot,2,2
Dale_avia,4,4
air_France,1,1
British_AW,6,16


# Задача 5.
Найти количество маршрутов, которые обслуживаются наибольшим числом рейсов.
Замечания.
* A - B и B - A считать разными маршрутами.
* Использовать только таблицу Trip 
![](images/planes.png)

### Pandas

In [168]:
# Решить одной командой
# Не допускается ипользование merge и дополнительных переменных

(trip.reset_index().groupby(['town_from', 'town_to']).nunique()['trip_no'] == 
 trip.reset_index().groupby(['town_from', 'town_to']).nunique()['trip_no'].max()).sum()


4

# Задача 6.
Определить номера пассажиров, которые больше других времени провели в полетах.
Вывести имена и общее время в минутах, проведенное в полетах

![](images/planes.png)

### Pandas

In [228]:
# Здесь расматриваются типы времени: изначально timestamp, и после взятия разности -- timediff.
# Обращение к методам времени осуществляется через поле series.dt

trip['flight_duration'] = (trip['time_in'] - trip['time_out']) / pd.Timedelta('1 minute')
psg_flight_time = pass_in_trip.reset_index().merge(trip, left_on='trip_no', right_index=True).groupby('id_psg')\
    .agg({'flight_duration': sum})
names_time = psg_flight_time[psg_flight_time['flight_duration'] == psg_flight_time['flight_duration'].max()]\
    .merge(passenger, left_index=True, right_index=True).set_index('name')
names_time

,flight_duration
name,
Michael Caine,2100.0
